1. Здійсніть імпорт необхідних пакетів.

In [3]:
!pip install category_encoders
!pip install gdown
import gdown
import warnings
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
import category_encoders as ce
from scipy.stats import skew, kurtosis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.5 MB/s eta 0:00:00


2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv набори даних, доступні в репозиторії. Цільова змінна — рівень заробітної плати співробітників (Salary).

In [30]:
url = 'https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ'
output = 'mod_04_hw_train_data.csv'
gdown.download(url, output, quiet=False)
df_train = pd.read_csv(output)

url_val = 'https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a'
output_val = 'mod_04_hw_valid_data.csv'
gdown.download(url_val, output_val, quiet=False)
df_val = pd.read_csv(output_val)

Downloading...
From: https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ
To: /content/mod_04_hw_train_data.csv
100%|██████████| 17.2k/17.2k [00:00<00:00, 24.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a
To: /content/mod_04_hw_valid_data.csv
100%|██████████| 549/549 [00:00<00:00, 1.42MB/s]


3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

In [31]:
numerical_features = ['Experience']
categorical_features = ['Qualification', 'Role', 'Cert']

In [32]:
columns_to_drop = ['Date_Of_Birth', 'University', 'Phone_Number', 'Name']

df_train = df_train.drop(columns=columns_to_drop)
df_val = df_val.drop(columns=columns_to_drop)

print("Remaining columns after dropping:")
print(df_train.columns)

print("Remaining  df_val columns after dropping:")
print(df_val.columns)

Remaining columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')
Remaining  df_val columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')


In [33]:
# Заповнення числових стовпців середнім значенням
for column in numerical_features:
    df_train[column] = df_train[column].fillna(df_train[column].mean())

# Заповнення категоріальних стовпців модою
for column in categorical_features:
    df_train[column] = df_train[column].fillna(df_train[column].mode()[0])

# Перевірка після заповнення
print(df_train.isnull().sum())

Experience       0
Qualification    0
Role             0
Cert             0
Salary           0
dtype: int64


In [34]:
# Очищення від викидів
# Метод IQR
Q1 = df_train['Experience'].quantile(0.25)
Q3 = df_train['Experience'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Фільтрація викидів
train_data = df_train[(df_train['Experience'] >= lower_bound) & (df_train['Experience'] <= upper_bound)]

# Перевірка розмірності
print("Розмірність після очищення:", df_train.shape)

Розмірність після очищення: (249, 5)


4. Виконайте обробку числових ознак (трансформацію / нормалізацію за допомогою об’єктів StandardScaler або PowerTransformer з пакета sklearn) і кодування категоріальних (наприклад, за допомогою об’єктів OneHotEncoder / TargetEncoder з пакета sklearn або об’єктів з пакета category_encoders).

In [35]:
numerical_features = ['Experience']
categorical_features = ['Qualification', 'Role', 'Cert']

X_train = df_train.drop('Salary', axis=1)
y_train = df_train['Salary']

scaler = StandardScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_features])

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical = encoder.fit_transform(X_train[categorical_features]).toarray()

X_train_transformed = np.hstack((X_train_numerical, X_train_categorical))

In [10]:

print(X_train_transformed[:5])

[[-0.29668344  0.          1.          0.          0.          1.
   0.          0.          1.        ]
 [ 1.04791858  0.          0.          1.          0.          0.
   1.          0.          1.        ]
 [ 1.04791858  0.          1.          0.          0.          0.
   1.          0.          1.        ]
 [-0.29668344  1.          0.          0.          0.          1.
   0.          1.          0.        ]
 [ 1.04791858  1.          0.          0.          0.          0.
   1.          0.          1.        ]]


5. Побудуйте модель за допомогою об'єкта KNeighborsRegressor з пакета sklearn.

In [36]:
# Ініціалізація моделі
model = KNeighborsRegressor(n_neighbors=4, weights='uniform', metric='manhattan')  # Вибір кількості сусідів

# Навчання моделі
model.fit(X_train_transformed, y_train)

# Прогнозування на тренувальному наборі
y_pred_train = model.predict(X_train_transformed)

# Обчислення MAPE
mape = mean_absolute_percentage_error(y_train, y_pred_train)

# Виведення результату
print(f'Обчислена MAPE: {mape:.2%}')

Обчислена MAPE: 3.56%


6. Виконайте аналогічні етапи обробки і підготовки даних для валідаційного набору, подібно до того, як ми це робили у темі «Метод опорних векторів (SVM)» в розділі «Практика застосування SVM-класифікатора. Навчання й оцінка моделі. Приклад використання моделі)» .

In [37]:
# Підготовка валідаційних даних
X_valid = df_val.drop('Salary', axis=1)
y_valid = df_val['Salary']

# Нормалізація числових ознак
X_valid_numerical = scaler.transform(X_valid[numerical_features])

# Кодування категоріальних ознак
X_valid_categorical = encoder.transform(X_valid[categorical_features]).toarray()

# Об'єднання числових і категоріальних ознак
X_valid_transformed = np.hstack((X_valid_numerical, X_valid_categorical))

# Прогнозування на валідаційному наборі
y_pred_valid = model.predict(X_valid_transformed)

# Обчислення MAPE для валідаційного набору
mape_valid = mean_absolute_percentage_error(y_valid, y_pred_valid)
print(f'MAPE для валідаційного набору: {mape_valid:.2%}')

MAPE для валідаційного набору: 13.90%


7. Отримайте прогноз заробітної плати для нових об’єктів (працівників) із валідаційного набору, розрахуйте доцільні метрики точності регресійної моделі.